# 1. Create a spark data frame that contains your favorite programming languages.

- The name of the column should be language
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [1]:
import pandas as pd
import numpy as np
import pyspark
import pyspark.sql.functions as F 

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/24 09:07:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:

pd_df = pd.DataFrame({'language': ['python', 'r',
 'scala', 'java', 'c', 'c++']})

In [4]:
pd_df

,language
0,python
1,r
2,scala
3,java
4,c
5,c++


In [5]:

df = spark.createDataFrame(pd_df)

In [6]:
df.show()

+--------+
|language|
+--------+
|  python|
|       r|
|   scala|
|    java|
|       c|
|     c++|
+--------+



In [7]:
df.schema

StructType([StructField('language', StringType(), True)])

In [8]:
df.count(), len(df.columns)

(6, 1)

In [9]:
df.describe().show()

+-------+--------+
|summary|language|
+-------+--------+
|  count|       6|
|   mean|    null|
| stddev|    null|
|    min|       c|
|    max|   scala|
+-------+--------+



In [10]:
df.show(5)

+--------+
|language|
+--------+
|  python|
|       r|
|   scala|
|    java|
|       c|
+--------+
only showing top 5 rows



# 1. Load the mpg dataset as a spark dataframe.

In [11]:
from pydataset import data
mpg_pandas = data('mpg')

In [12]:
mpg = spark.createDataFrame(mpg_pandas)


In [14]:
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows




- Create 1 column of output that contains a message like the one below:

In [15]:

mpg.select(
    F.concat(
    F.lit('The '), 
    mpg.year,
    F.lit(' '), 
    mpg.manufacturer,
    F.lit(' has a '),
    mpg.cyl,
    F.lit(' cylinder engine')).alias('description')
    ).show(truncate=False)

+------------------------------------------+
|description                               |
+------------------------------------------+
|The 1999 audi has a 4 cylinder engine     |
|The 1999 audi has a 4 cylinder engine     |
|The 2008 audi has a 4 cylinder engine     |
|The 2008 audi has a 4 cylinder engine     |
|The 1999 audi has a 6 cylinder engine     |
|The 1999 audi has a 6 cylinder engine     |
|The 2008 audi has a 6 cylinder engine     |
|The 1999 audi has a 4 cylinder engine     |
|The 1999 audi has a 4 cylinder engine     |
|The 2008 audi has a 4 cylinder engine     |
|The 2008 audi has a 4 cylinder engine     |
|The 1999 audi has a 6 cylinder engine     |
|The 1999 audi has a 6 cylinder engine     |
|The 2008 audi has a 6 cylinder engine     |
|The 2008 audi has a 6 cylinder engine     |
|The 1999 audi has a 6 cylinder engine     |
|The 2008 audi has a 6 cylinder engine     |
|The 2008 audi has a 8 cylinder engine     |
|The 2008 chevrolet has a 8 cylinder engine|
|The 2008 

 - Transform the `trans` column so that it only contains either `manual` or `auto`.

In [16]:
mpg.select(mpg.trans).show(10)

+----------+
|     trans|
+----------+
|  auto(l5)|
|manual(m5)|
|manual(m6)|
|  auto(av)|
|  auto(l5)|
|manual(m5)|
|  auto(av)|
|manual(m5)|
|  auto(l5)|
|manual(m6)|
+----------+
only showing top 10 rows



In [17]:
mpg.withColumn('tran', 
               F.when(
                mpg.trans.like('auto%'), 'auto'
                ).otherwise(
                    'manual'
                    )
            ).select('trans','tran').show(5)

+----------+------+
|     trans|  tran|
+----------+------+
|  auto(l5)|  auto|
|manual(m5)|manual|
|manual(m6)|manual|
|  auto(av)|  auto|
|  auto(l5)|  auto|
+----------+------+
only showing top 5 rows



# Load the tips dataset as a spark dataframe.

## 1. What percentage of observations are smokers?

In [18]:

# load up tips from pydataset, feed it into createDataFrame
tips = spark.createDataFrame(data('tips'))


In [19]:
tips.count()

244

In [20]:
tips.groupby('smoker').count().show()

+------+-----+
|smoker|count|
+------+-----+
|    No|  151|
|   Yes|   93|
+------+-----+



In [21]:
# group by smoker column,
# grab the counts of each subpopulation,
# make a new column (withColumn) called percent
# reference the new aggreagted column count, divide by the length of the df
# multiply by 100 to get the percentage, round the whole thing
# then show
tips.groupby('smoker').count().withColumn(
    'percent', F.round(
        F.col('count') / tips.count() * 100
        )
        ).show()

+------+-----+-------+
|smoker|count|percent|
+------+-----+-------+
|    No|  151|   62.0|
|   Yes|   93|   38.0|
+------+-----+-------+



# 2. 2. Create a column that contains the tip percentage

In [22]:
tips.columns

['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [23]:
tips.withColumn(
    'tip_percentage', tips.tip / tips.total_bill
    ).show(5)


+----------+----+------+------+---+------+----+-------------------+
|total_bill| tip|   sex|smoker|day|  time|size|     tip_percentage|
+----------+----+------+------+---+------+----+-------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|0.05944673337257211|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|0.16054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|0.16658733936220846|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 0.1397804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|0.14680764538430255|
+----------+----+------+------+---+------+----+-------------------+
only showing top 5 rows



In [24]:
tips.select(
    tips.tip,
    tips.total_bill,
    F.round(
        (tips.tip / tips.total_bill), 4
        ).alias('tip_percentage')
        ).show(5)

+----+----------+--------------+
| tip|total_bill|tip_percentage|
+----+----------+--------------+
|1.01|     16.99|        0.0594|
|1.66|     10.34|        0.1605|
| 3.5|     21.01|        0.1666|
|3.31|     23.68|        0.1398|
|3.61|     24.59|        0.1468|
+----+----------+--------------+
only showing top 5 rows



3. Calculate the average tip percentage for each combination of sex and smoker.

In [25]:
# make the same tip_percentage column that I just did
# from that point, pass a groupby
# sex, smoker
# pass aggregative function mean to tip_percentage (applied to each group)
tips.withColumn(
    'tip_percentage',
    tips.tip / tips.total_bill
    ).groupby(
    'sex',
    'smoker').agg(
        F.round(
            F.mean('tip_percentage'),4).alias(
                'avg_tip_p')
                ).show()

+------+------+---------+
|   sex|smoker|avg_tip_p|
+------+------+---------+
|  Male|    No|   0.1607|
|Female|    No|   0.1569|
|  Male|   Yes|   0.1528|
|Female|   Yes|   0.1822|
+------+------+---------+



In [26]:
# pivot version:
tips.groupby(
    'sex').pivot(
        'smoker').agg(
            F.round(F.mean(tips.tip / tips.total_bill),4)).show()

+------+------+------+
|   sex|    No|   Yes|
+------+------+------+
|Female|0.1569|0.1822|
|  Male|0.1607|0.1528|
+------+------+------+



# 1. Use the seattle weather dataset referenced in the lesson to answer the questions below.

## - Convert the temperatures to fahrenheit.

In [ ]:
from vega_datasets import data

weather = data.seattle_weather()
weather = spark.createDataFrame(weather)